# 🧮 Algebraic Language Model Exploration

## Learning Objectives
By the end of this notebook, you will:
- Master the algebraic operators for composing language models (+, *, |, &, ^, **, >>, <<, ~)
- Understand how to combine n-gram models with different orders
- Learn to apply projections for context transformation
- Create sophisticated model compositions
- Benchmark and visualize model performance

## Prerequisites
- Basic understanding of language models and n-grams
- Python programming experience
- Familiarity with probability distributions

## Estimated Time: 45 minutes

## 📚 Part 1: Setup and Imports

First, let's import all necessary modules and set up our environment.

In [ ]:
# Core imports
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Optional, Tuple
import time
import json
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

# Configure visualization
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Import our algebraic framework
try:
    from src.model_algebra import (
        LanguageModel, NGramModel, MixtureModel,
        WeightedModel, UnionModel, IntersectionModel,
        ComplementModel, PowerModel, SequenceModel,
        XORModel
    )
    print("✅ Successfully imported algebraic framework")
except ImportError as e:
    print(f"⚠️ Import error: {e}")
    print("Using fallback implementations...")
    # Fallback definitions
    from ngram_projections.models.base import LanguageModel
    from ngram_projections.models.ngram import NGramModel

## 🎯 Part 2: Understanding the Algebraic Operators

Our framework provides 10+ algebraic operators for composing language models:

| Operator | Symbol | Description | Example |
|----------|--------|-------------|----------|
| Addition | `+` | Equal-weight mixture | `model1 + model2` |
| Multiplication | `*` | Weighted scaling | `0.3 * model` |
| Union | `\|` | Maximum probability | `model1 \| model2` |
| Intersection | `&` | Minimum probability | `model1 & model2` |
| XOR | `^` | Exclusive or | `model1 ^ model2` |
| Power | `**` | Model self-mixture | `model ** 2` |
| Composition | `>>` | Sequential application | `model1 >> model2` |
| Reverse Comp | `<<` | Reverse sequential | `model1 << model2` |
| Complement | `~` | Invert probabilities | `~model` |
| Projection | `@` | Apply transformation | `model @ projection` |

In [ ]:
# Interactive demonstration of operators
def visualize_operator(op_name, op_symbol, description, example):
    """Create a visual card for an operator."""
    print(f"╔{'═'*60}╗")
    print(f"║ {op_name:20} {op_symbol:^10} {' '*28}║")
    print(f"╠{'═'*60}╣")
    print(f"║ {description:58} ║")
    print(f"║ Example: {example:48} ║")
    print(f"╚{'═'*60}╝")

# Display operator reference
operators = [
    ("Addition", "+", "Equal-weight mixture of models", "model1 + model2"),
    ("Multiplication", "*", "Scale model contribution", "0.3 * model"),
    ("Union", "|", "Max probability per token", "model1 | model2"),
    ("Intersection", "&", "Min probability per token", "model1 & model2"),
    ("XOR", "^", "Exclusive combination", "model1 ^ model2"),
]

for op in operators[:3]:  # Show first 3
    visualize_operator(*op)
    print()

## 🏗️ Part 3: Building and Training N-gram Models

Let's create a diverse training corpus and build n-gram models of different orders.

In [ ]:
# Create comprehensive training data
training_corpus = {
    "scientific": [
        "quantum computing leverages quantum mechanics for computation",
        "neural networks learn hierarchical representations from data",
        "machine learning algorithms optimize objective functions",
        "deep learning models require substantial computational resources",
        "artificial intelligence systems exhibit intelligent behavior",
    ],
    "literary": [
        "the quick brown fox jumps over the lazy dog",
        "in the garden roses bloom under moonlight",
        "ancient wisdom speaks through forgotten manuscripts",
        "the river flows gently through the valley",
        "stars illuminate the darkness of night",
    ],
    "technical": [
        "the server processes requests using multiple threads",
        "database queries optimize through indexed columns",
        "api endpoints handle authentication and authorization",
        "cloud infrastructure scales automatically under load",
        "microservices communicate through message queues",
    ]
}

# Flatten corpus
all_texts = []
for category, texts in training_corpus.items():
    all_texts.extend(texts)
    print(f"📖 {category.capitalize()}: {len(texts)} documents")

print(f"\n📊 Total training documents: {len(all_texts)}")

In [ ]:
# Train n-gram models with different orders
def train_ngram_models(texts, orders=[2, 3, 4]):
    """Train n-gram models of different orders."""
    models = {}
    stats = {}
    
    for n in orders:
        print(f"\n🔧 Training {n}-gram model...")
        model = NGramModel(n=n)
        
        # Train on all texts
        total_ngrams = 0
        for text in texts:
            tokens = text.lower().split()
            model.train(tokens)
            total_ngrams += max(0, len(tokens) - n + 1)
        
        models[n] = model
        stats[n] = {
            'unique_ngrams': len(model.counts),
            'total_ngrams': total_ngrams,
            'avg_frequency': total_ngrams / len(model.counts) if model.counts else 0
        }
        
        print(f"  ✅ Unique {n}-grams: {stats[n]['unique_ngrams']:,}")
        print(f"  ✅ Total {n}-grams: {stats[n]['total_ngrams']:,}")
        print(f"  ✅ Avg frequency: {stats[n]['avg_frequency']:.2f}")
    
    return models, stats

# Train models
ngram_models, ngram_stats = train_ngram_models(all_texts, [2, 3, 4, 5])

# Visualize statistics
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Unique n-grams
orders = list(ngram_stats.keys())
unique_counts = [ngram_stats[n]['unique_ngrams'] for n in orders]
ax1.bar(orders, unique_counts, color='skyblue', edgecolor='navy')
ax1.set_xlabel('N-gram Order')
ax1.set_ylabel('Unique N-grams')
ax1.set_title('Model Complexity')
ax1.grid(True, alpha=0.3)

# Average frequency
avg_freq = [ngram_stats[n]['avg_frequency'] for n in orders]
ax2.plot(orders, avg_freq, 'o-', color='coral', linewidth=2, markersize=8)
ax2.set_xlabel('N-gram Order')
ax2.set_ylabel('Average Frequency')
ax2.set_title('Data Sparsity')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 🧪 Part 4: Basic Algebraic Operations - Hands-on Experiments

Now let's experiment with different algebraic operations and see how they affect predictions.

In [ ]:
# Interactive prediction explorer
class PredictionExplorer:
    """Interactive tool for exploring model predictions."""
    
    def __init__(self, models):
        self.models = models
    
    def predict_and_visualize(self, context, models_to_test=None, top_k=5):
        """Predict and visualize results from multiple models."""
        if models_to_test is None:
            models_to_test = list(self.models.items())
        
        fig, axes = plt.subplots(1, len(models_to_test), figsize=(4*len(models_to_test), 5))
        if len(models_to_test) == 1:
            axes = [axes]
        
        print(f"🔍 Context: '{' '.join(context)}'\n")
        
        for ax, (name, model) in zip(axes, models_to_test):
            # Get predictions
            preds = model.predict(context)
            
            if preds:
                # Get top k predictions
                top_preds = sorted(preds.items(), key=lambda x: x[1], reverse=True)[:top_k]
                tokens = [t for t, _ in top_preds]
                probs = [p for _, p in top_preds]
                
                # Visualize
                bars = ax.bar(range(len(tokens)), probs, color='steelblue', edgecolor='navy')
                ax.set_xticks(range(len(tokens)))
                ax.set_xticklabels(tokens, rotation=45, ha='right')
                ax.set_ylabel('Probability')
                ax.set_title(name, fontweight='bold')
                ax.set_ylim([0, max(probs) * 1.2 if probs else 1])
                
                # Add value labels on bars
                for bar, prob in zip(bars, probs):
                    height = bar.get_height()
                    ax.text(bar.get_x() + bar.get_width()/2., height,
                           f'{prob:.3f}', ha='center', va='bottom', fontsize=8)
                
                # Print top prediction
                print(f"  {name}: '{tokens[0]}' ({probs[0]:.3f})")
            else:
                ax.text(0.5, 0.5, 'No predictions', ha='center', va='center', 
                       transform=ax.transAxes)
                ax.set_title(name, fontweight='bold')
        
        plt.suptitle(f"Model Predictions for: '{' '.join(context)}'", fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()

# Create explorer
explorer = PredictionExplorer(ngram_models)

In [ ]:
# Experiment 1: Pure n-gram models
print("🔬 Experiment 1: Comparing N-gram Orders\n")
print("How does n-gram order affect predictions?\n")

test_context = ["machine", "learning"]

models_to_compare = [
    ("2-gram", ngram_models[2]),
    ("3-gram", ngram_models[3]),
    ("4-gram", ngram_models[4]),
]

explorer.predict_and_visualize(test_context, models_to_compare)

In [ ]:
# Experiment 2: Model Addition (Mixture)
print("🔬 Experiment 2: Model Addition (Equal Mixture)\n")
print("What happens when we combine models with + operator?\n")

# Create mixtures
mixture_2_3 = ngram_models[2] + ngram_models[3]
mixture_3_4 = ngram_models[3] + ngram_models[4]
mixture_all = ngram_models[2] + ngram_models[3] + ngram_models[4]

models_to_compare = [
    ("2-gram", ngram_models[2]),
    ("3-gram", ngram_models[3]),
    ("2+3 mixture", mixture_2_3),
    ("2+3+4 mixture", mixture_all),
]

explorer.predict_and_visualize(test_context, models_to_compare)

In [ ]:
# Experiment 3: Weighted Mixtures
print("🔬 Experiment 3: Weighted Mixtures\n")
print("How do weights affect the mixture?\n")

# Try different weight configurations
weight_configs = [
    (0.9, 0.1, "90% 2-gram + 10% 3-gram"),
    (0.5, 0.5, "50% 2-gram + 50% 3-gram"),
    (0.1, 0.9, "10% 2-gram + 90% 3-gram"),
]

weighted_models = []
for w1, w2, name in weight_configs:
    model = w1 * ngram_models[2] + w2 * ngram_models[3]
    weighted_models.append((name, model))

# Add pure models for reference
weighted_models.insert(0, ("Pure 2-gram", ngram_models[2]))
weighted_models.append(("Pure 3-gram", ngram_models[3]))

# Visualize
test_context = ["neural", "networks"]
explorer.predict_and_visualize(test_context, weighted_models[:4])

## 🎛️ Part 5: Advanced Operators - Interactive Playground

Let's explore the more advanced operators: Union (|), Intersection (&), XOR (^), and Power (**).

In [ ]:
# Create specialized models for demonstration
scientific_model = NGramModel(n=3)
literary_model = NGramModel(n=3)

# Train on specialized corpora
for text in training_corpus["scientific"]:
    scientific_model.train(text.lower().split())

for text in training_corpus["literary"]:
    literary_model.train(text.lower().split())

print("✅ Trained specialized models:")
print(f"  Scientific model: {len(scientific_model.counts)} n-grams")
print(f"  Literary model: {len(literary_model.counts)} n-grams")

In [ ]:
# Interactive operator comparison
def compare_operators(model1, model2, context, operators=['|', '&', '^']):
    """Compare different operators on two models."""
    
    results = {}
    
    # Get individual predictions
    pred1 = model1.predict(context)
    pred2 = model2.predict(context)
    
    # Apply operators
    if '|' in operators:  # Union
        union_model = model1 | model2
        results['Union (|)'] = union_model.predict(context)
    
    if '&' in operators:  # Intersection
        inter_model = model1 & model2
        results['Intersection (&)'] = inter_model.predict(context)
    
    if '^' in operators:  # XOR
        xor_model = model1 ^ model2
        results['XOR (^)'] = xor_model.predict(context)
    
    if '+' in operators:  # Addition
        add_model = model1 + model2
        results['Addition (+)'] = add_model.predict(context)
    
    # Visualize
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    # Plot individual models
    for i, (name, pred) in enumerate([('Scientific', pred1), ('Literary', pred2)]):
        if pred:
            tokens = list(pred.keys())[:5]
            probs = [pred[t] for t in tokens]
            axes[i].bar(tokens, probs, color='lightblue' if i == 0 else 'lightcoral')
            axes[i].set_title(f'{name} Model', fontweight='bold')
            axes[i].set_ylabel('Probability')
            axes[i].set_xticklabels(tokens, rotation=45)
    
    # Plot operator results
    colors = ['green', 'orange', 'purple', 'red']
    for i, (op_name, pred) in enumerate(results.items()):
        ax = axes[i + 2]
        if pred:
            tokens = list(pred.keys())[:5]
            probs = [pred[t] for t in tokens]
            ax.bar(tokens, probs, color=colors[i % len(colors)])
            ax.set_title(f'{op_name}', fontweight='bold')
            ax.set_ylabel('Probability')
            ax.set_xticklabels(tokens, rotation=45)
    
    # Hide unused subplot
    if len(results) < 4:
        axes[-1].axis('off')
    
    plt.suptitle(f"Operator Comparison | Context: '{' '.join(context)}'", 
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    return results

# Test advanced operators
test_context = ["the", "system"]
print("🔬 Advanced Operator Comparison\n")
results = compare_operators(scientific_model, literary_model, test_context, 
                           operators=['|', '&', '^', '+'])

## 🔄 Part 6: Context Transformations and Projections

Projections transform the context before feeding it to the model. Let's implement and test various projections.

In [ ]:
# Implement custom projections
class Projection:
    """Base class for context projections."""
    def project(self, context: List[str]) -> List[str]:
        raise NotImplementedError

class RecencyProjection(Projection):
    """Keep only recent tokens."""
    def __init__(self, max_length=3):
        self.max_length = max_length
    
    def project(self, context: List[str]) -> List[str]:
        return context[-self.max_length:]

class LowercaseProjection(Projection):
    """Convert to lowercase."""
    def project(self, context: List[str]) -> List[str]:
        return [t.lower() for t in context]

class StopwordFilterProjection(Projection):
    """Remove common stopwords."""
    def __init__(self):
        self.stopwords = {'the', 'a', 'an', 'is', 'are', 'was', 'were', 'be', 'been'}
    
    def project(self, context: List[str]) -> List[str]:
        return [t for t in context if t.lower() not in self.stopwords]

class KeywordBoostProjection(Projection):
    """Duplicate important keywords."""
    def __init__(self, keywords):
        self.keywords = set(keywords)
    
    def project(self, context: List[str]) -> List[str]:
        result = []
        for token in context:
            result.append(token)
            if token.lower() in self.keywords:
                result.append(token)  # Duplicate keyword
        return result

# Create projections
recency_proj = RecencyProjection(max_length=3)
lowercase_proj = LowercaseProjection()
stopword_proj = StopwordFilterProjection()
keyword_proj = KeywordBoostProjection(['quantum', 'neural', 'learning'])

print("✅ Created 4 different projection types")

In [ ]:
# Visualize projection effects
def visualize_projections(context, projections):
    """Show how different projections transform context."""
    
    print("🔄 Context Transformation Visualization\n")
    print(f"Original context: {context}")
    print(f"Length: {len(context)}\n")
    print("="*60)
    
    results = []
    for name, proj in projections:
        transformed = proj.project(context)
        results.append((name, transformed))
        
        print(f"\n📐 {name}:")
        print(f"  Result: {transformed}")
        print(f"  Length: {len(transformed)}")
        
        # Show changes
        if len(transformed) < len(context):
            print(f"  ⬇️ Reduced by {len(context) - len(transformed)} tokens")
        elif len(transformed) > len(context):
            print(f"  ⬆️ Increased by {len(transformed) - len(context)} tokens")
        else:
            print(f"  ↔️ Same length")
    
    return results

# Test projections
test_context = ["The", "neural", "network", "learns", "from", "the", "training", "data"]

projections = [
    ("Recency (last 3)", recency_proj),
    ("Lowercase", lowercase_proj),
    ("Stopword Filter", stopword_proj),
    ("Keyword Boost", keyword_proj),
]

results = visualize_projections(test_context, projections)

In [ ]:
# Apply projections to models
class ProjectedModel(LanguageModel):
    """Model with projection applied."""
    
    def __init__(self, model, projection):
        self.model = model
        self.projection = projection
    
    def predict(self, context):
        transformed = self.projection.project(context)
        return self.model.predict(transformed)
    
    def train(self, tokens):
        self.model.train(tokens)

# Create projected models
base_model = ngram_models[3]
recency_model = ProjectedModel(base_model, recency_proj)
filtered_model = ProjectedModel(base_model, stopword_proj)

# Compare predictions
test_context = ["the", "machine", "learning", "algorithm", "processes"]

print("🔬 Projection Effects on Predictions\n")

models_to_test = [
    ("No Projection", base_model),
    ("Recency Projection", recency_model),
    ("Stopword Filtered", filtered_model),
]

for name, model in models_to_test:
    preds = model.predict(test_context)
    if preds:
        top_pred = max(preds.items(), key=lambda x: x[1])
        print(f"{name:20} → Top: '{top_pred[0]}' ({top_pred[1]:.3f})")
    else:
        print(f"{name:20} → No predictions")

## ⚡ Part 7: Performance Benchmarking

Let's benchmark different model configurations to understand their performance characteristics.

In [ ]:
# Comprehensive benchmarking suite
class ModelBenchmark:
    """Benchmark model performance."""
    
    def __init__(self):
        self.results = {}
    
    def benchmark(self, models, test_contexts, runs=100):
        """Run performance benchmarks."""
        
        for name, model in models:
            times = []
            memory = []
            
            for _ in range(runs):
                # Time prediction
                start = time.time()
                for context in test_contexts:
                    _ = model.predict(context)
                elapsed = time.time() - start
                times.append(elapsed * 1000)  # Convert to ms
            
            self.results[name] = {
                'mean_time': np.mean(times),
                'std_time': np.std(times),
                'min_time': np.min(times),
                'max_time': np.max(times),
            }
        
        return self.results
    
    def visualize(self):
        """Visualize benchmark results."""
        if not self.results:
            print("No results to visualize")
            return
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        
        # Bar chart of mean times
        names = list(self.results.keys())
        means = [self.results[n]['mean_time'] for n in names]
        stds = [self.results[n]['std_time'] for n in names]
        
        colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(names)))
        bars = ax1.bar(range(len(names)), means, yerr=stds, 
                      color=colors, capsize=5, edgecolor='black')
        ax1.set_xticks(range(len(names)))
        ax1.set_xticklabels(names, rotation=45, ha='right')
        ax1.set_ylabel('Time (ms)')
        ax1.set_title('Model Latency Comparison', fontweight='bold')
        ax1.grid(True, alpha=0.3)
        
        # Add value labels
        for bar, mean in zip(bars, means):
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height,
                    f'{mean:.2f}', ha='center', va='bottom', fontsize=9)
        
        # Box plot for distribution
        data_for_box = []
        labels_for_box = []
        for name in names:
            # Simulate distribution
            mean = self.results[name]['mean_time']
            std = self.results[name]['std_time']
            samples = np.random.normal(mean, std, 100)
            data_for_box.append(samples)
            labels_for_box.append(name)
        
        bp = ax2.boxplot(data_for_box, labels=labels_for_box, patch_artist=True)
        for patch, color in zip(bp['boxes'], colors):
            patch.set_facecolor(color)
        ax2.set_ylabel('Time (ms)')
        ax2.set_title('Latency Distribution', fontweight='bold')
        ax2.grid(True, alpha=0.3)
        ax2.set_xticklabels(labels_for_box, rotation=45, ha='right')
        
        plt.tight_layout()
        plt.show()
        
        # Print summary table
        print("\n📊 Performance Summary")
        print("="*70)
        print(f"{'Model':<25} {'Mean (ms)':<12} {'Std (ms)':<12} {'Min (ms)':<12} {'Max (ms)':<12}")
        print("-"*70)
        
        for name in names:
            r = self.results[name]
            print(f"{name:<25} {r['mean_time']:<12.3f} {r['std_time']:<12.3f} "
                  f"{r['min_time']:<12.3f} {r['max_time']:<12.3f}")

# Run benchmarks
benchmark = ModelBenchmark()

# Models to benchmark
models_to_benchmark = [
    ("2-gram", ngram_models[2]),
    ("3-gram", ngram_models[3]),
    ("4-gram", ngram_models[4]),
    ("2+3 mixture", ngram_models[2] + ngram_models[3]),
    ("Weighted (0.3*2 + 0.7*3)", 0.3 * ngram_models[2] + 0.7 * ngram_models[3]),
    ("Union (2|3)", ngram_models[2] | ngram_models[3]),
]

# Test contexts
test_contexts = [
    ["the", "model"],
    ["neural", "network", "learns"],
    ["machine", "learning", "algorithm", "processes"],
]

print("⏱️ Running performance benchmarks...")
results = benchmark.benchmark(models_to_benchmark, test_contexts, runs=50)
benchmark.visualize()

## 🎯 Part 8: Practical Application - Building a Domain-Specific Model

Let's build a sophisticated model for a specific use case using our algebraic framework.

In [ ]:
# Build a technical documentation assistant
class TechnicalAssistant:
    """Domain-specific model using algebraic composition."""
    
    def __init__(self):
        # Train specialized models
        self.api_model = NGramModel(n=3)
        self.error_model = NGramModel(n=3)
        self.general_model = NGramModel(n=2)
        
        # Training data
        api_docs = [
            "the api endpoint returns json response",
            "authentication token must be included in headers",
            "rate limiting prevents excessive requests",
            "the rest api uses standard http methods",
        ]
        
        error_docs = [
            "error code 404 indicates resource not found",
            "internal server error requires investigation",
            "timeout errors occur under heavy load",
            "validation error means invalid input data",
        ]
        
        # Train models
        for doc in api_docs:
            self.api_model.train(doc.split())
        
        for doc in error_docs:
            self.error_model.train(doc.split())
        
        for doc in api_docs + error_docs:
            self.general_model.train(doc.split())
        
        # Create composite model
        self.model = self._build_composite_model()
    
    def _build_composite_model(self):
        """Build sophisticated composite model."""
        # API-focused mixture
        api_mixture = 0.7 * self.api_model + 0.3 * self.general_model
        
        # Error-focused mixture
        error_mixture = 0.8 * self.error_model + 0.2 * self.general_model
        
        # Combined with union for fallback
        combined = (0.5 * api_mixture + 0.5 * error_mixture) | self.general_model
        
        return combined
    
    def complete(self, context, mode='auto'):
        """Get completions with mode selection."""
        
        # Auto-detect mode from context
        if mode == 'auto':
            context_str = ' '.join(context).lower()
            if 'error' in context_str or 'exception' in context_str:
                mode = 'error'
            elif 'api' in context_str or 'endpoint' in context_str:
                mode = 'api'
            else:
                mode = 'general'
        
        # Select appropriate model
        if mode == 'api':
            model = self.api_model
        elif mode == 'error':
            model = self.error_model
        else:
            model = self.model  # Use composite
        
        return model.predict(context), mode

# Create assistant
assistant = TechnicalAssistant()

# Test different contexts
test_cases = [
    (["the", "api", "endpoint"], 'auto'),
    (["error", "code"], 'auto'),
    (["the", "server"], 'auto'),
    (["authentication", "token"], 'api'),
    (["timeout", "errors"], 'error'),
]

print("🤖 Technical Documentation Assistant\n")
print("="*60)

for context, mode_hint in test_cases:
    preds, detected_mode = assistant.complete(context, mode_hint)
    
    print(f"\nContext: {' '.join(context)}")
    print(f"Mode: {detected_mode}")
    
    if preds:
        top_3 = sorted(preds.items(), key=lambda x: x[1], reverse=True)[:3]
        print("Suggestions:")
        for token, prob in top_3:
            print(f"  • {token} ({prob:.3f})")

## 🔍 Part 9: Model Introspection and Analysis

Let's analyze how our algebraic operations affect model behavior internally.

In [ ]:
# Analyze probability distributions
def analyze_distribution(model, context, name="Model"):
    """Analyze the probability distribution of a model."""
    
    preds = model.predict(context)
    
    if not preds:
        print(f"No predictions for {name}")
        return None
    
    # Calculate statistics
    probs = list(preds.values())
    stats = {
        'num_tokens': len(preds),
        'max_prob': max(probs),
        'min_prob': min(probs),
        'mean_prob': np.mean(probs),
        'std_prob': np.std(probs),
        'entropy': -sum(p * np.log2(p) if p > 0 else 0 for p in probs),
        'top_token': max(preds.items(), key=lambda x: x[1])[0],
    }
    
    return stats

# Compare distributions
def compare_distributions(models, context):
    """Compare probability distributions across models."""
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    axes = axes.flatten()
    
    stats_list = []
    
    for i, (name, model) in enumerate(models[:6]):
        ax = axes[i]
        
        preds = model.predict(context)
        stats = analyze_distribution(model, context, name)
        
        if preds and stats:
            stats_list.append((name, stats))
            
            # Plot distribution
            tokens = list(preds.keys())[:10]
            probs = [preds[t] for t in tokens]
            
            bars = ax.bar(range(len(tokens)), probs, color='steelblue')
            ax.set_xticks(range(len(tokens)))
            ax.set_xticklabels(tokens, rotation=45, ha='right', fontsize=8)
            ax.set_ylabel('Probability')
            ax.set_title(f"{name}\nEntropy: {stats['entropy']:.2f}", fontsize=10)
            
            # Add entropy line
            ax.axhline(y=1/len(preds), color='red', linestyle='--', 
                      alpha=0.5, label='Uniform')
        else:
            ax.text(0.5, 0.5, 'No predictions', ha='center', va='center',
                   transform=ax.transAxes)
            ax.set_title(name)
    
    # Hide unused axes
    for i in range(len(models), 6):
        axes[i].axis('off')
    
    plt.suptitle(f"Distribution Analysis | Context: '{' '.join(context)}'", 
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Print statistics table
    if stats_list:
        print("\n📊 Distribution Statistics")
        print("="*80)
        print(f"{'Model':<20} {'Entropy':<10} {'Max Prob':<10} {'Top Token':<15} {'# Tokens':<10}")
        print("-"*80)
        
        for name, stats in stats_list:
            print(f"{name:<20} {stats['entropy']:<10.3f} {stats['max_prob']:<10.3f} "
                  f"{stats['top_token']:<15} {stats['num_tokens']:<10}")

# Test distribution analysis
context = ["machine", "learning"]

models_to_analyze = [
    ("2-gram", ngram_models[2]),
    ("3-gram", ngram_models[3]),
    ("Equal Mix", ngram_models[2] + ngram_models[3]),
    ("Weighted Mix", 0.2 * ngram_models[2] + 0.8 * ngram_models[3]),
    ("Union", ngram_models[2] | ngram_models[3]),
    ("Intersection", ngram_models[2] & ngram_models[3]),
]

compare_distributions(models_to_analyze, context)

## 💡 Part 10: Advanced Techniques and Best Practices

Let's explore advanced techniques for optimal model composition.

In [ ]:
# Best practices demonstration
print("🎓 Best Practices for Algebraic Model Composition\n")
print("="*60)

# 1. Hierarchical Composition
print("\n1️⃣ Hierarchical Composition")
print("-"*40)
print("Build complex models from simple components:")

# Base models
base_2 = ngram_models[2]
base_3 = ngram_models[3]
base_4 = ngram_models[4]

# Level 1: Simple mixtures
level1_a = 0.6 * base_2 + 0.4 * base_3
level1_b = 0.3 * base_3 + 0.7 * base_4

# Level 2: Combine mixtures
level2 = (level1_a | level1_b)  # Union for robustness

# Level 3: Final composition
final = 0.8 * level2 + 0.2 * base_2  # Add fallback

print("  Base → Mixtures → Union → Final")
print(f"  Complexity: {3} levels")

# 2. Dynamic Weight Adjustment
print("\n2️⃣ Dynamic Weight Adjustment")
print("-"*40)

def dynamic_mixture(models, context):
    """Adjust weights based on context length."""
    context_len = len(context)
    
    if context_len < 2:
        # Short context: prefer lower-order
        weights = [0.8, 0.2, 0.0]
    elif context_len < 4:
        # Medium context: balanced
        weights = [0.3, 0.5, 0.2]
    else:
        # Long context: prefer higher-order
        weights = [0.1, 0.3, 0.6]
    
    # Create weighted mixture
    result = None
    for model, weight in zip(models, weights):
        if weight > 0:
            weighted = weight * model
            result = weighted if result is None else result + weighted
    
    return result, weights

# Test dynamic adjustment
test_contexts = [
    ["the"],
    ["machine", "learning"],
    ["the", "neural", "network", "model", "learns"],
]

models = [base_2, base_3, base_4]

for ctx in test_contexts:
    mixture, weights = dynamic_mixture(models, ctx)
    print(f"  Context length {len(ctx)}: weights = {weights}")

# 3. Ensemble Strategies
print("\n3️⃣ Ensemble Strategies")
print("-"*40)

# Voting ensemble
def voting_ensemble(models, context, threshold=0.1):
    """Combine models through voting."""
    votes = Counter()
    
    for model in models:
        preds = model.predict(context)
        if preds:
            # Vote for tokens above threshold
            for token, prob in preds.items():
                if prob > threshold:
                    votes[token] += 1
    
    # Normalize to probabilities
    total = sum(votes.values())
    if total > 0:
        return {token: count/total for token, count in votes.items()}
    return {}

# Test voting
context = ["neural", "network"]
ensemble_pred = voting_ensemble([base_2, base_3, base_4], context)

if ensemble_pred:
    top_token = max(ensemble_pred.items(), key=lambda x: x[1])
    print(f"  Voting ensemble prediction: '{top_token[0]}' ({top_token[1]:.3f})")

# 4. Performance Optimization
print("\n4️⃣ Performance Optimization Tips")
print("-"*40)
print("  • Cache frequently used mixtures")
print("  • Use Union (|) for parallel search")
print("  • Apply projections to reduce context size")
print("  • Precompute static model combinations")
print("  • Profile to identify bottlenecks")

## 🎯 Part 11: Your Turn - Interactive Playground

Use this section to experiment with your own model compositions!

In [ ]:
# Interactive model builder
print("🛠️ Interactive Model Builder\n")
print("Build your own model using algebraic operations!\n")
print("Available base models: base_2, base_3, base_4")
print("Available operators: +, *, |, &, ^\n")

# Example compositions to try
examples = [
    "0.5 * base_2 + 0.5 * base_3",
    "(base_2 | base_3) & base_4",
    "0.3 * base_2 + 0.4 * base_3 + 0.3 * base_4",
    "base_2 ^ base_3",
]

print("Example compositions to try:")
for i, example in enumerate(examples, 1):
    print(f"  {i}. {example}")

# TODO: Modify this section to create your own compositions
# Your composition here:
my_model = 0.4 * base_2 + 0.6 * base_3  # Change this!

# Test your model
test_contexts = [
    ["machine", "learning"],
    ["neural", "network"],
    ["the", "model"],
]

print("\n🧪 Testing your model:")
print("="*40)

for context in test_contexts:
    preds = my_model.predict(context)
    if preds:
        top = max(preds.items(), key=lambda x: x[1])
        print(f"Context: {' '.join(context):20} → '{top[0]}' ({top[1]:.3f})")

## 📋 Summary and Key Takeaways

### What We've Learned

1. **Algebraic Operators**: 
   - Addition (+) for equal mixtures
   - Multiplication (*) for weighted scaling
   - Union (|) for maximum/fallback behavior
   - Intersection (&) for consensus
   - XOR (^) for exclusive combination

2. **Projections**:
   - Transform context before model processing
   - Can filter, boost, or reshape input
   - Composable with >> operator

3. **Performance**:
   - Simple models are fast but less accurate
   - Complex compositions trade speed for quality
   - Caching and optimization are crucial

4. **Best Practices**:
   - Start simple, add complexity gradually
   - Use hierarchical composition
   - Consider dynamic weight adjustment
   - Profile and optimize bottlenecks

### Next Steps

1. Try the **lightweight_grounding_demo.ipynb** to see real-world applications
2. Explore **unified_algebra.ipynb** for advanced theoretical concepts
3. Build your own domain-specific models
4. Experiment with custom projections
5. Benchmark against traditional approaches

### 🚀 Challenge Yourself

Can you create a model that:
- Adapts to context length dynamically?
- Combines 5+ different n-gram orders efficiently?
- Uses projections to improve accuracy by 20%?
- Achieves <1ms latency for predictions?

Happy experimenting! 🎉